In [ ]:
pip install langchain tiktoken nltk ipykernel
pip install langchain-text-splitters
python -m ipykernel install --user --name=limpeza_ds

In [2]:

from pathlib import Path
import os

SRC_GLOB = "corpus_*.jsonl"
OUT_SUFFIX = "_clean"

VLLM_URL = os.environ.get("VLLM_URL", "http://localhost:8000/v1/chat/completions")
MODEL_ID = os.environ.get("VLLM_MODEL", "Qwen/Qwen2.5-32B-Instruct")

# ATENÇÃO: Aumentado de 2048 (muito baixo) para 16384.
# Garanta que seu servidor vLLM foi iniciado com --max-model-len 16384 ou mais.
MAX_NEW_TOKENS = 16384
TEMPERATURE = 0.01
TOP_P = 1.0

# --- NOVAS CONFIGURAÇÕES DE CHUNKING ---
# Tamanho alvo para os chunks de saída (em tokens estimados)
TARGET_CHUNK_TOKENS = 8000
# Sobreposição (em tokens) para manter o contexto entre chunks
CHUNK_OVERLAP_TOKENS = 500
# Mínimo de caracteres para um chunk ser salvo (evita chunks vazios)
MIN_CHUNK_LEN = 100


MODEL_TOTAL_CONTEXT = 32768 

CONTEXT_SAFETY_BUFFER = 2048

print("Endpoint:", VLLM_URL)
print("Modelo (informativo):", MODEL_ID)
print("Padrão de entrada:", SRC_GLOB)
print(f"Tamanho alvo do Chunk: {TARGET_CHUNK_TOKENS} tokens")

Endpoint: http://localhost:8000/v1/chat/completions
Modelo (informativo): Qwen/Qwen2.5-32B-Instruct
Padrão de entrada: corpus_*.jsonl
Tamanho alvo do Chunk: 8000 tokens


In [3]:
# Dependências


import json, re, time
import requests
from tqdm import tqdm
from glob import glob

# --- NOVOS IMPORTS para Chunking e Estimativa de Tokens ---
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
import nltk

# --- INICIALIZAÇÃO DOS NOVOS UTILS ---
# Garante que o NLTK (usado pelo LangChain) tenha o 'punkt' para sentenças
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Baixando 'punkt' do NLTK para divisão de sentenças...")
    nltk.download('punkt', quiet=True)

# Inicializa o 'cl100k_base' (usado por GPT-4 e Qwen) para estimar tokens
TOKEN_COUNTER = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    """Estima o número de tokens usando tiktoken."""
    # disallowed_special=() previne erros em tokens especiais como <|endoftext|>
    return len(TOKEN_COUNTER.encode(text, disallowed_special=()))

# --- FIM DOS NOVOS UTILS ---

session = requests.Session()
session.headers.update({"Authorization": "Bearer dummy"})

SYS_PROMPT = """Você é um assistente de curadoria de dados de alta precisão, especializado em Processamento de Linguagem Natural. Sua única tarefa é extrair, de forma 100% verbatim, o conteúdo principal (título e prosa) do texto bruto fornecido, eliminando todo o "boilerplate".

Siga estas regras rigorosamente:

### 1. Regras de Extração (O que manter)
* **Extração Verbatim:** Mantenha o texto do artigo principal exatamente como está. Não resuma, não parafraseie, não corrija erros de digitação e não altere nenhuma palavra.
* **Manter Títulos:** Mantenha o título principal e os subtítulos (headings) que separam as seções do artigo, pois eles fazem parte do conteúdo.

### 2. Regras de Remoção (O que eliminar)
Elimine *completamente* os seguintes elementos:
* Datas e horas de publicação (ex: "28 de janeiro de 2025, 7h02").
* Listas de referências, bibliografias e notas de rodapé (ex: "[1] MENDES...", "[2] CONVENÇÃO...", etc.).
* Citações numéricas no meio do texto (ex: "no texto [1]", "a lei [5]", "o autor [11].").
* Citações acadêmicas e referências no formato (AUTOR, ANO), (AUTOR et al., ANO), ou semelhantes, inclusive citações múltiplas, como (Silva, 2023; Mendes, 2022).
* Títulos repetidos.
* URLs e datas de acesso (ex: "Disponível em: https://...", "Acesso em: 23 de janeiro...").
* Biografias de autor (ex: "Sobre o Autor", "Escrito por Fulano...").
* Menus de navegação, links de redes sociais, "Artigos Relacionados", "Leia Também".
* Frases de call-to-action (ex: "Encontrou um erro?", "Avise nossa equipe", "Comente abaixo", "Clique aqui").
* Qualquer texto que seja claramente parte do layout do site e não do conteúdo do artigo.
* Trechos ou linhas que contenham apenas pontilhados (ex: '....................'), pois representam texto omitido.
* Artigos ou cláusulas de vigência (ex: 'Esta Lei entra em vigor...') e de revogação (ex: 'Revogam-se as disposições em contrário.', 'Fica revogada a Lei nº X...').
* Artigos ou parágrafos vetados ou revogados (ex: 'Art. 12. (Revogado)', 'Art. 13. (VETADO)').

### 3. Formato de Saída (CRÍTICO)
* Sua resposta deve conter *apenas* o texto limpo extraído.
* Não inclua o texto original na resposta.*
* Não inclua *nenhum* preâmbulo, saudação ou explicação (ex: "Claro, aqui está o texto limpo:", "Texto extraído:", "== bruto ==", "Saída", etc.).
* Apenas o texto extraído e nada mais.
* Ajuste a formatação do texto para que não contenha frases em maiúsculo, numeração repetida e quebra de linhas extras.
* Não remova partes do conteúdo. Este tem que estar na íntegra.
* Não inclua nenhum caracter ou parâmetro de controle  no final do texto (ex: "<|endoftext|>")."""

FOOTER_RE = re.compile(
    r"(Encontrou um erro|Compartilhe|Imprimir|Leia também|Política de privacidade)", 
    re.IGNORECASE
)

VIGENCIA_RE = re.compile(
    r'(^\s*Art\.?\s*[\dº§]+\.?\s*)?\s*Esta (Lei|Emenda|Portaria|Resolução).*entra em vigor', 
    re.IGNORECASE
)

# Regex para artigos de REVOGAÇÃO (explícita)
# (Opcional "Art. X.") + "Fica(m) revogada(s)" ou "Revoga(m)-se"
REVOGACAO_RE = re.compile(
    r'(^\s*Art\.?\s*[\dº§]+\.?\s*)?\s*(Fica(m)? revogada(s)?|Revoga(m)?-se)', 
    re.IGNORECASE
)

REVOGACAO_FINAL_RE = re.compile(
    r'^\s*Revogam-se as disposições em contrário', 
    re.IGNORECASE
)

def post_rules(txt: str) -> str:
    """
    Aplica regras de limpeza de Regex (pós-LLM) para remover ruídos
    acadêmicos e legislativos.
    """
    
    # --- 1. Normalização Inicial (Global) ---
    txt = txt.replace("-\n", "") 
    txt = re.sub(r"[ \t]+", " ", txt) 
    txt = re.sub(r"\n{3,}", "\n\n", txt) 

    # --- 2. Limpeza Linha-a-Linha ---
    cleaned_lines = [] 
    
    for line in txt.splitlines():
        
        # Regra A: Pular linhas que são *APENAS* pontilhados
        if re.match(r'^\s*[\.·…]{3,}\s*$', line):
            continue
            
        # Regra B: Remover pontilhados *INLINE* (no meio)
        line = re.sub(r'[\.·…]{3,}', ' ', line)
        
        # Regra C: Remover pontilhados do *FINAL* da linha (redundante, mas seguro)
        line = re.sub(r'[\.·…]{3,}$', '', line)

        # Regra D: Pular linhas que são *apenas* (Revogado) ou (VETADO)
        if re.match(r'^\s*\((Revogado|VETADO)\).*\s*$', line, re.I):
            continue
            
        # Regra E: Pular linhas de rodapé (ex: "Compartilhe")
        if FOOTER_RE.search(line):
            continue
                
        # Regra F (ATUALIZADA): Pular artigos de Vigência e Revogação
        if VIGENCIA_RE.search(line) or REVOGACAO_RE.search(line) or REVOGACAO_FINAL_RE.search(line):
            continue

        # Normaliza o espaço em branco extra
        line = re.sub(r'\s+', ' ', line).strip()
        
        if line:
            cleaned_lines.append(line)
            
    txt = "\n".join(cleaned_lines)
    
    # --- 3. Limpeza Pós-Junção (Citações Acadêmicas) ---
    citation_regex = r"\s\((?!.*(Lei|Art\.|Decreto|CF|nº))[A-ZÁÉÍÓÚÂÊÔÃÕÜÇ][^)]*(19|20)\d{2}[^)]*\)"
    txt = re.sub(citation_regex, " ", txt, flags=re.IGNORECASE)
    
    return txt.strip()
    

def chat_clean(text: str, max_new_tokens: int = 5000, timeout: int = 600) -> str:
    # ATENÇÃO: Esta função falhará se 'text' (entrada) for > 32k tokens.
    # O notebook não trata isso; ele assume que a entrada cabe no contexto.
    payload = {
        "model": MODEL_ID,
        "temperature": TEMPERATURE,
        "top_p": TOP_P,
        "max_tokens": max_new_tokens,
        "messages": [
            {"role": "system", "content": SYS_PROMPT},
            {"role": "user", "content": text}
        ]
    }
    r = session.post(VLLM_URL, json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"].strip()

print("Dependências e funções prontas.")

Dependências e funções prontas.


In [4]:
# Função Limpar Arquivo

def clean_file(src_path: str, dst_path: str, sample: int = 0):
    total = 0; ok = 0; t_in = 0; t_out = 0; errs = 0; chunks_total = 0

    # --- INICIALIZA O SPLITTER (FATIAMENTO) ---
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=TARGET_CHUNK_TOKENS,
        chunk_overlap=CHUNK_OVERLAP_TOKENS,
        length_function=count_tokens,
        separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""] # Lista explícita
    )

    with open(src_path, "r", encoding="utf-8") as fin, open(dst_path, "w", encoding="utf-8") as fout:
        if sample > 0:
            def head(f, n):
                for i, line in enumerate(f):
                    if i >= n: break
                    yield line
            lines = head(fin, sample)
        else:
            lines = fin
        
        # Define as posições corretas das barras de progresso (Discutido anteriormente)
        pbar_files = tqdm(files, desc="Progresso Geral (Arquivos)", position=0, leave=True)
        # (Se este for o loop principal, mova esta linha para a Célula 4)
        
        for line in tqdm(lines, desc=f"Limpando {src_path}", position=1, leave=False):
            total += 1
            try:
                obj = json.loads(line)
                raw = obj.get("text", "")
                if not isinstance(raw, str) or len(raw.strip()) < 50:
                    continue

                # --- INÍCIO DA CORREÇÃO DO ERRO 400 (Contexto Dinâmico) ---
                
                # 1. Estimar tokens de entrada
                input_tokens = count_tokens(raw)
                
                # 2. Verificar se a ENTRADA sozinha já excede o limite
                if input_tokens >= MODEL_TOTAL_CONTEXT - CONTEXT_SAFETY_BUFFER:
                    tqdm.write(f"AVISO [{src_path}]: Texto de ENTRADA ({input_tokens} tokens) excede o limite. Descartando.")
                    errs += 1
                    continue

                # 3. Calcular espaço de saída disponível
                available_output_tokens = MODEL_TOTAL_CONTEXT - input_tokens - CONTEXT_SAFETY_BUFFER
                
                # 4. Definir o max_tokens final
                # Usamos o MÍNIMO entre o que queremos (DEFAULT_MAX_OUTPUT_TOKENS)
                # e o que realmente cabe (available_output_tokens).
                final_max_tokens = min(MAX_NEW_TOKENS, available_output_tokens)

                # 5. Verificar se temos espaço de saída suficiente
                if final_max_tokens <= MIN_CHUNK_LEN: # (MIN_CHUNK_LEN da Célula 1)
                     tqdm.write(f"AVISO [{src_path}]: Sem espaço de saída ({final_max_tokens} tokens). Descartando.")
                     errs += 1
                     continue

                # 6. Chamar a limpeza com o valor dinâmico
                t_in += len(raw)
                cleaned = chat_clean(raw, max_new_tokens=final_max_tokens)
                
                # --- FIM DA CORREÇÃO ---
                
                cleaned = post_rules(cleaned)
                
                if len(cleaned) < 50:
                    continue

                # --- LÓGICA DE CHUNKING ---
                num_tokens = count_tokens(cleaned)

                if num_tokens <= TARGET_CHUNK_TOKENS:
                    t_out += len(cleaned)
                    fout.write(json.dumps({"text": cleaned + "\n\n<|endoftext|>\n"}, ensure_ascii=False) + "\n")
                    chunks_total += 1
                else:
                    chunks = text_splitter.split_text(cleaned)
                    for chunk in chunks:
                        if len(chunk) < MIN_CHUNK_LEN:
                            continue
                        t_out += len(chunk)
                        fout.write(json.dumps({"text": chunk + "\n\n<|endoftext|>\n"}, ensure_ascii=False) + "\n")
                        chunks_total += 1
                
                ok += 1
                
            except Exception as e:
                # Modificado para imprimir o erro específico
                tqdm.write(f"ERRO INESPERADO em {src_path}: {e}")
                errs += 1
                time.sleep(0.25)
                
    return {
        "arquivo": src_path, "total_linhas": total, "gravadas_ok": ok, 
        "total_chunks_gerados": chunks_total, "erros": errs,
        "chars_in": t_in, "chars_out": t_out,
        "reducao_%": round(100 * (1 - (t_out / max(1, t_in))), 2)
    }

print("Função pronta.")

Função pronta.


In [ ]:
# Teste Processamento

texto_teste = """
ENCONTROU UM ERRO? avise nossa equipe
Renata Costa do Egyto ACADÊMICA DE DIREITO NA UNICATÓLICA DE TOCANTINS\nIgor de Andrade Barbosa DEFENSOR PÚBLICO FEDERAL E PROFESSOR\nLEI QUE AFASTA AGRESSOR DO AMBIENTE DOMÉSTICO TRANSFERE À POLÍCIA ATRIBUIÇÕES DO JUDICIÁRIO E NÃO ATENDE DEMANDAS ESPECÍFICAS DAS MULHERES VITIMADAS\nNa década de 1980, as reivindicações dos movimentos sociais, em particular dos movimentos feministas, foram centradas na necessidade de enfrentamento mais efetivo em relação à violência contra a mulher, com a criação das delegacias especializadas de atendimento à mulher (DEAM), responsáveis pelo registro e apuração de casos de violência de gênero. A polícia foi a primeira instituição que passou a enfrentar e, com isso, expor a problemática da agressão contra a mulher ao longo da história.\nA percepção das relações sociais de poder sobre o prisma do gênero permitiu a identificação de desigualdades, discriminação e violência contra as mulheres, embora sejam ainda hoje relativizadas.\nA Lei 11.340/06, denominada Lei Maria da Penha (LMP), é vista no Brasil como um marco na prevenção e no combate aos abusos contra a mulher, com penas mais rigorosas, trazendo a expectativa de efetiva punição para os agressores, e medidas protetivas e de assistência às vítimas. No entanto, as estatísticas da violência doméstica e familiar no Brasil continuam sendo um desafio para os especialistas. Com 15 anos de vigência da lei, os registros evidenciam significativa elevação, ano após ano, em todo o país.\nNa busca por maior efetividade no combate à truculência doméstica, editou-se a Lei 13.827/19, que, entre outras disposições, trouxe a possibilidade de aplicação das medidas protetivas de urgência (MPU) pela autoridade policial, inovação apoiada por grande parte da doutrina, porém contando com posicionamentos contrários.\nDiante desse cenário, o presente estudo busca reunir os argumentos favoráveis e contrários à aplicação das MPU pela autoridade policial.\n1. O SISTEMA BRASILEIRO DE PROTEÇÃO À MULHER E A LEI MARIA DA PENHA\nComo dito inicialmente, a LMP estabeleceu mecanismos para prevenir, punir e erradicar a violência doméstica e familiar contra a mulher. O nome da lei se deu em homenagem a Maria da Penha Maia Fernandes, cearense, biofarmacêutica, que sofreu agressões físicas e psicológicas de seu cônjuge. Após duas tentativas de homicídio, em 1983 Maria da Penha levou a conhecimento público a brutalidade por que passou, resultando na condenação do agressor, em 1996, pela Justiça brasileira, depois de o primeiro julgamento ter sido anulado em 1991.\nEm 1998, mesmo com o trânsito em julgado, a execução da pena imposta ao, agora, seu ex-marido não havia sido iniciada. Em razão disso, Maria da Penha apresentou petição à Comissão Interamericana de Direitos Humanos (CIDH) com o apoio do Centro para Justiça e\nDireito Internacional (CEJIL) e do Comitê Latino-Americano e do Caribe para a Defesa dos Direitos da Mulher (CLADEM).\nEm agosto daquele ano, a CEJIL recebeu a denúncia que evidenciava a inércia do Estado brasileiro perante as atrocidades sofridas por Maria da Penha, pois, passados 15 anos dos fatos que culminaram na condenação do agressor, medidas efetivas para processá-lo e puni-lo ainda não haviam sido tomadas.\nA comissão, reconhecendo a negligência praticada pelo Brasil no caso, notificou o Estado brasileiro e, por fim, recomendou, entre outras medidas, a adoção de providências com a finalidade de disponibilizar às demais vítimas de violência doméstica meios mais céleres para processar as denúncias e punir os agressores.\nEmbora o relatório contivesse algumas ações necessárias a serem tomadas pelo governo, não houve de sua parte resposta à comissão e, em razão disso, no ano de 2001 foi emitido o Relatório 54, responsabilizando o Brasil por sua negligência e omissão no combate à violência contra as mulheres.\nPor fim, a comissão condenou o estado do Ceará (onde ocorreu o crime) ao pagamento de indenização de R$ 20.000,00 por não ter punido o réu, valor que foi pago devidamente corrigido. Além disso, o ex-marido foi preso um ano após a decisão da CIDH, em 2002, para cumprir a pena de dez anos de prisão, que ocorreu somente após 19 anos e 6 meses da ocorrência do crime e seis meses antes de sua prescrição.\nO relatório da CIDH expôs a posição omissa do Estado-parte, restando configurada violação à Convenção de Belém do Pará, ratificada pelo Brasil em 1984, tendo sido o primeiro país a ser condenado em âmbito internacional pela violação à proteção dos direitos humanos. A Lei Maria da Penha, portanto, foi um avanço, levando em consideração que o Brasil previa em sua legislação termos como “mulher honesta” e tolerava situações de agressão em defesa da honra masculina.\nA intervenção do Estado na esfera doméstica apresenta fases distintas, podendo se dar mediante intervenção preventiva primária ou pós-conflitual. A primeira hipótese busca prevenir casos de violência, objetivando evitar a sua concretização. Exerce papel fundamental no enfrentamento do problema, pois, conforme ensina Beccaria (1998, p. 130), “é melhor prevenir os delitos do que puni-los”, e aloca-se no domínio privilegiado da educação, do esclarecimento e da sensibilização para o respeito e a proteção dos direitos fundamentais.\nAssumindo sua função preventiva e assistencial (CUNHA; PINTO, 2007), a LMP determina que as políticas públicas coíbam a violência doméstica ou familiar contra as mulheres por meio de ações articuladas da União, estados, Distrito Federal e municípios, e de ações não governamentais, tendo como diretriz “a integração operacional do Poder Judiciário, do Ministério Público e da Defensoria Pública com as áreas de segurança pública, assistência social, saúde, educação, trabalho e habitação” (art. 8º, I).\nA exemplo, cita-se o art. 9º da Lei 11.340/06, que prevê as normas de assistência (recorrentes aos princípios e diretrizes previstos na LOAS, no Sistema Único de Saúde – SUS – e no Sistema Único de Segurança Pública – SUSP), tratando da inclusão da mulher no cadastro de programas assistenciais; remoção de servidora pública; manutenção de vínculo trabalhista.\nAo ingressar no campo das ações efetivas, o legislador faz referência à LOAS (Lei 8.742/93), cujas diretrizes principais estão em seu art. 2º1, dispostas de forma mais genérica, deixando de indicar, objetivamente, as fontes de recursos e instrumentos disponíveis para viabilizar a “política pública” citada no art. 8º, referindo-se apenas que a assistência “será prestada de forma articulada e conforme os princípios e diretrizes” previstos em leis que já estão em vigor, mas que não têm sido eficazes na redução dos problemas sociais que afligem a sociedade, mormente aqueles decorrentes da pobreza.\nO SUS e o SUSP têm sua parcela de contribuição a oferecer, porém já estão demasiadamente sobrecarregados, principalmente o último, que praticamente não saiu do papel e não vem cumprindo as suas funções. O SUS deve ser aperfeiçoado para cumprir a sua função na política de combate à violência doméstica e familiar e de apoio à vítima, ao passo que o SUSP precisa se firmar como órgão articulador das ações a serem desenvolvidas nas esferas federal, estadual e municipal na área da segurança pública.\nA LMP impõe como dever do Estado-juiz a determinação de proteger a mulher por meio de programas assistenciais. Contudo, para que isso surta seus efeitos, é preciso que os poderes Legislativo e Executivo cumpram seus deveres em todas as esferas, instituindo tais programas e viabilizando verbas orçamentárias necessárias para seu pleno funcionamento, sob pena da prestação jurisdicional ser ineficaz (SOUZA, 2019).\nAtualmente, mulheres de todas as camadas sociais podem contar com delegacias especializadas, casas-abrigo, centros de apoio e outros programas governamentais voltados para a garantia de seus direitos e defesa de sua segurança e integridade. Entretanto, ainda há problemas no cumprimento das leis, resultando em baixa efetividade das ações oferecidas para proteção, reabilitação e inclusão, demonstrando resquícios da cultura patriarcal, machista e misógina presente na sociedade (BIGLIARDI; ANTUNES, 2018).\nComo a questão requer medidas urgentes, repressivas e preventivas compatíveis com cada uma das situações que envolvam relações íntimas ou familiares, há uma multiplicidade de medidas específicas no art. 22 e, ainda, um extenso rol de medidas não especificadas, autorizando a vara competente a deferi-las de acordo com o caso concreto.\nSituações complexas que se apresentem como desafiadoras à criatividade do aplicador são, por exemplo, aquelas em que a vítima e o suposto agressor trabalham no mesmo local, frequentam o mesmo templo religioso ou vivem no local ou próximo de onde exercem suas atividades profissionais. O juiz deverá verificar se há para a vítima risco suficiente para justificar a necessidade da imposição de medida que inviabilize o agressor de continuar trabalhando, decidindo a qual dos dois cabe impor o afastamento, mormente porque poderá resultar na demissão ou na redução dos seus ganhos, em prejuízo do sustento da própria família (SOUZA, 2019).\nO art. 22 apresenta rol exemplificativo de medidas protetivas de urgência (MPU), as quais possuem características e natureza próprias, a maioria delas podendo, inclusive, ser aplicadas tanto no âmbito criminal quanto no cível e mesmo no juízo de família, quando a matéria não for da competência do juizado especializado previsto no art. 14, ou da vara criminal mencionada no art. 33. As medidas protetivas podem, inclusive, ser aplicadas com as medidas cautelares inseridas nos arts. 319 e 320 do Código de Processo Penal (CPP), alterados pela Lei 12.403/11, quando estiverem presentes os requisitos necessários.\nUma das medidas mais frequentemente imposta é o afastamento do lar, consistindo na retirada do acusado do local em que convive com a ofendida, uma vez que tal medida se mostra necessária para evitar que a vítima seja submetida a coação psicológica, principalmente nos casos de relação conjugal (CUNHA; PINTO, 2007). Trata-se, portanto, de medida similar já existente no ordenamento jurídico, inserida no trecho final do parágrafo único do art. 69 da Lei 9.099/95, pela Lei 10.455/02, além de guardar semelhança com o “afastamento temporário de um dos cônjuges da morada do casal” (BRASIL, 1995).\nCabe destacar que a Lei 13.827/19 possibilitou a imposição do afastamento do agressor pelo delegado de polícia e até por policiais civis e militares, em caráter pré-cautelar, quando preenchidos os requisitos estabelecidos no art. 12-C da LMP, devendo o juiz competente ser comunicado “no prazo máximo de 24 horas”, a quem cabe decidir “em igual prazo, sobre a manutenção ou a revogação da medida aplicada, devendo dar ciência ao Ministério Público concomitantemente” (BRASIL, 2006).\nJá o art. 23 da LMP traz em seu escopo cinco2 distintas medidas protetivas visando salvaguardar seus objetivos, especialmente no que se refere à garantia da integridade moral, física, psicológica e material da mulher vitimada pela brutalidade no seio do lar e da família.\nPor fim, o art. 24 possibilita que o juiz do Juizado de Violência Doméstica e Familiar contra a Mulher conceda medidas protetivas de natureza eminentemente patrimonial em favor da vítima, voltadas a proteger o patrimônio comum do casal.\nEm sede conclusiva, é possível afirmar que a LMP representou grande avanço no enfrentamento da criminalidade doméstica contra as mulheres no Brasil, reforçando a tutela penal e trazendo instrumentos protetivos de grande valoração para as ofendidas, a ponto de gerar maior sensação de segurança às vítimas (MONTENEGRO, 2015).\nNo entanto, a lei não foi suficiente para reduzir de forma significativa os índices deste tipo de barbárie.\n2. ARGUMENTOS FAVORÁVEIS ÀS ALTERAÇÕES DA LEI 13.827/19\nA Lei Maria da Penha não surgiu somente com o objetivo de enfrentar a violência doméstica e familiar contra a mulher, mas também como instrumento de prevenção à opressão de gênero e de assistência. A fim de acelerar e aprimorar os efeitos das medidas de proteção, foi editada a Lei 13.827/19 (BONFIM, 2021), que alterou a LMP, acrescentando-lhe o art. 12-C:\nArt. 12-C. Verificada a existência de risco atual ou iminente à vida ou à integridade física da mulher em situação de violência doméstica e familiar, ou de seus dependentes, o agressor será imediatamente afastado do lar, domicílio ou local de convivência com a ofendida:\nI − pela autoridade judicial;\nII − pelo delegado de polícia, quando o Município não for sede de comarca; ou\nIII − pelo policial, quando o Município não for sede de comarca e não houver delegado disponível no momento da denúncia.\n§ 1º. Nas hipóteses dos incisos II e III do caput deste artigo, o juiz será comunicado no prazo máximo de 24 (vinte e quatro) horas e decidirá, em igual prazo, sobre a manutenção ou a revogação da medida aplicada, devendo dar ciência ao Ministério Público concomitantemente.\n§ 2º. Nos casos de risco à integridade física da ofendida ou à efetividade da medida protetiva de urgência, não será concedida liberdade provisória ao preso.\nSegundo Leitão e Oliveira (2019), a inovação legislativa que permitiu a aplicação das medidas protetivas de urgência pelo delegado de polícia e por policiais civis e militares foi um avanço na proteção às mulheres. Isso porque os agressores eram beneficiados pela liberdade provisória, resultando em mais agressões, muitas delas tendo por consequência a morte da vítima. Assim, no entendimento dos autores, se a medida se mostrar eficaz, a tendência é que os óbitos e eventos de prepotência contra a mulher sejam reduzidos.\nOs mesmos autores argumentam que, ao ampliar o rol de agentes públicos autorizados a conferir essa medida, a nova lei prestigia e assegura maior segurança às vítimas, trazendo a possibilidade de maior economia ao erário público (LEITÃO; OLIVEIRA, 2019).\nSantos (2019) também reputa que a inovação foi positiva, pois a mudança beneficia municípios que não são sedes de comarcas e delegacias em que, no momento da denúncia, o delegado não estiver presente.\nGarcia (2019), referindo-se à ADIN 6138, ajuizada no STF, questiona a competência do delegado ou a dos policiais para aplicar a medida de afastamento do agressor, pois entende que a tese utilizada não pode prosperar pois, embora preveja a possibilidade administrativa para a concessão de MPU – a exemplo de como já é feito com a lavratura do auto de prisão em flagrante e com o relaxamento do flagrante pelo delegado de polícia –, não é retirada a palavra final do magistrado. A MPU é apenas antecipada, como já é feito no caso de flagrante, em que se permite que qualquer pessoa dê voz de prisão a quem estiver cometendo um delito.\nNo mesmo sentido é o entendimento de Cunha (2019), ao afirmar que os agentes policiais praticam em seu dia a dia atos que limitam direitos fundamentais dos cidadãos e que isso não pode servir de justificativa para se cogitar que seja arguida a inconstitucionalidade dessa prática, até porque a decisão tomada pelo delegado de polícia ou por outros agentes da segurança não é soberana, devendo ser submetida a um juiz em 24 horas para que possa dar – ou não – o seu aval.\nPercebe-se, pois, que o que realmente se quis privilegiar foi a dignidade da pessoa humana, o mais relevante princípio dentre todos os demais. Ademais, o delegado de polícia detém os conhecimentos necessários para avaliar se a medida protetiva é ou não necessária.\nNão obstante as razões favoráveis ao afastamento do agressor do lar pela autoridade policial, há também argumentos contrários à nova legislação, igualmente abalizados e que serão detalhados a seguir.\n3. ARGUMENTOS CONTRÁRIOS À LEI 13.827/19\nA Associação Nacional dos Membros do Ministério Público (CONAMP, 2016), durante as discussões do projeto de Lei 13.827/19, emitiu nota técnica dispondo sobre o fato de a carta magna de 1988 ter estabelecido um sistema de garantia de direitos fundamentais, através do qual algumas restrições de direitos precisavam ser submetidas a prévia decisão judicial.\nSegundo o parecer, o Supremo Tribunal Federal (STF) tem reiterado o denominado princípio da reserva de jurisdição3, aplicável a todas as restrições impostas aos direitos sensíveis, a exemplo da limitação à liberdade e outras medidas de natureza cautelar ou investigativas.\nO CONAMP (2016) concluiu pela inconstitucionalidade da transferência, mesmo que provisória, do poder de limitar os direitos fundamentais afetos à esfera judicial para a esfera policial, levando-se em conta que a opção pela aplicação das MPU se consubstancia em ingerência grave nos direitos fundamentais do acusado.\nMüller (2019), por sua vez, argumenta que a concessão de MPU pela autoridade policial pode ser entendida como um dispositivo pertencente ao direito penal, uma vez que o autor a enxerga como uma forma a mais de assegurar proteção à mulher. Dessa forma, o autor a contempla como uma forma de impelir o Estado a formular políticas públicas para prevenir e erradicar a violência doméstica de gênero no Brasil. O abuso contra a mulher, contudo, demanda tratamento realista, e não um que vise apenas atender aos clamores sociais, imputando à autoridade policial um poder decisório que até então era jurisdicional.\nOutra contradição, segundo Carvalho e Maia (2020), está na ineficiência da aplicação da decisão, por razões operacionais, já que no Brasil existem delegacias de polícia que sofrem com a falta de estrutura e recursos humanos para fazer frente às demandas atuais. Dessa forma, a atribuição adicional de cumprir as MPU, até então dos oficiais de justiça, torna os serviços prestados pela polícia ainda mais deficientes. Assim, se o objetivo é conferir maior efetividade à LMP, bastaria que fosse disponibilizada ao Judiciário melhor estrutura e um número maior de servidores, pois a mera transferência de determinadas prerrogativas às delegacias de polícia exige que a estrutura seja também necessária no órgão policial. Dessa forma, a Lei 13.827/19 está intencionalmente direcionada a atender às demandas sociais, já que não traz modificações significativas no campo jurídico, pois a aplicabilidade das MPU é a mesma, alterando-se somente os órgãos competentes para deferir e cumprir as medidas.\nOutro argumento citado por Müller (2019) é que, antes de ser aprovada, não foi realizado estudo estatístico que relacionasse o controle da violência doméstica com o deferimento de MPU pela autoridade policial, o que permitiria que a inconstitucionalidade dessa legislação fosse suscitada pela ausência de prognose legislativa. O que se observa, nesse caso, é um déficit na análise prognóstica de efetividade da alteração legislativa, havendo também deficiência nos estudos dos impactos da alteração sobre a violação a direitos fundamentais e reserva de jurisdição e da inafastabilidade do controle jurisdicional.\nJá Young (2016) questiona o interesse político dos delegados de polícia na aprovação das MPU, uma vez que a função de delegado já foi considerada uma carreira jurídica, o que abre margem para discutir a isonomia salarial dessa classe profissional em comparação com as demais carreiras da área, tendo em vista que a busca por atribuições de natureza jurídico-decisória seria um passo a mais para alavancar a carreira de delegado de polícia.\nDestaque-se que as dificuldades que assolam o Judiciário no Brasil atingem também as delegacias. Assim, da mesma forma que há falta de estrutura e recursos humanos no Judiciário, essas carências também são encontradas nas delegacias de polícia. De qualquer modo, tanto os oficiais de justiça como os agentes de polícia enfrentam dificuldades, especialmente em se tratando de localizar e citar o acusado (MÜLLER, 2019).\nTem-se que, não obstante a capacidade técnica e teórica dos delegados de polícia, como bem expõe Müller (2019), não se pode pactuar com a ideia de tornar os juízes meros homologadores de decisões proferidas por delegados. Os papéis se inverteriam, deslocando-se a competência para analisar a necessidade de conceder a MPU do Judiciário para a esfera policial, ainda que em casos específicos e de forma residual.\nAdemais, sabe-se que as MPU possuem efeitos eminentemente penais ou civis e, normalmente, restringem garantias fundamentais do cidadão, a exemplo do direito de ir e vir. Então, as MPU, especialmente as previstas no art. 22, inc. III, da Lei 11.340/06, só podem ser vistas como medidas válidas e idôneas se forem adequadamente fundamentadas em razões de fato e de direito e após se proceder à análise do caso concreto (CARVALHO; MAIA, 2020). Nessas hipóteses, o Judiciário atua como instância revisora.\nNo entanto, segundo Souza (2019), o que ocorre, na prática, é o desvio da função principal da polícia, que poderá ter que deixar de se dedicar a investigações criminais para cumprir mandados de intimação referentes à concessão de MPU. Transfere-se a atribuição dos oficiais de justiça para agentes policiais sem que estes últimos contem com a estrutura adequada e treinamento para assumir a nova função.\nPor fim, cita-se a ausência de preparo e de estrutura da Polícia Civil para atender às mulheres vítimas de violência, já que os agentes policiais não têm conhecimento sobre o desenvolvimento feminista da criminologia crítica, sendo possível observar o tratamento machista e patriarcal muitas vezes dispensado à vítima.\nDo exposto, com relação aos argumentos contrários, depreende-se que a concessão de MPU pela polícia não só é um meio ineficaz para proteger as mulheres, mas também majora o despotismo exercido contra elas, sendo, assim, uma estratégia excludente.\nCONCLUSÃO\nA hostilidade criminosa contra a mulher é um mal presente na sociedade, gerando consequências graves, que por muito tempo foram subestimadas. No entanto, foi visto que a Lei Maria da Penha tirou da invisibilidade os crimes de violência contra a mulher, prevendo mecanismos preventivos e repressivos que as empoderaram, estimulando-as a denunciar seus agressores e rompendo, dessa forma, com o ciclo da humilhação e agressividade.\nComo visto neste artigo, um dos mecanismos é a medida protetiva de urgência, voltada ao combate repressivo para fins de coibir a prática de crimes ultrajantes contra as mulheres. Porém, apesar desse esforço legislativo, as pesquisas demonstram que tais casos de violência têm aumentado. Nesse contexto, foi editada a Lei 13.827/19, que, entre outras disposições, trouxe a possibilidade de a autoridade policial conceder MPU, mais especificamente o afastamento do agressor do âmbito doméstico.\nSabe-se que a defesa da mulher demanda a real efetividade da LMP no que se refere ao atendimento e encaminhamento dos pedidos judiciais, desde as primeiras ameaças realizadas. Assim, entende-se que a simples mudança da esfera na qual serão concedidas as MPU não resolverá as dificuldades advindas da aplicação dessas políticas.\nSão insuficientes as delegacias especializadas de atendimento à mulher, refletindo na ausência de estrutura das delegacias de polícia e das equipes interdisciplinares responsáveis pelo atendimento à mulher agredida. A mudança legislativa que concedeu poderes judiciais\nàs autoridades policiais dará maior poder de decisão aos agentes, que não estarão, porém, auxiliados por equipe técnica, comprometendo a efetividade do atendimento no caso concreto.\nDo exposto, conclui-se que a aplicação das MPU pela autoridade policial em casos de violência doméstica contra a mulher viola o princípio da reserva da jurisdição, tendo em vista que transfere para a polícia atribuições que são do Judiciário, revelando, adicionalmente, o não atendimento a demandas específicas das mulheres vitimadas e a ingerência nos direitos fundamentais do acusado.\nREFERÊNCIAS\nBECCARIA, Cesare. Dos delitos e das penas. Tradução de Lucia Guidicini, Alessandro Berti Contessa. São Paulo: Martins Fontes, 1998.\nBIGLIARDI, Adriana Maria; ANTUNES, Maria Cristina. Violência contra mulheres. Curitiba: Juruá Editora, 2018.\nBONFIM, Katia Silveira. Lei 13.827/2019 altera a Lei Maria da Penha para permitir ao Delegado de Polícia a concessão de medida protetiva de urgência. Conteúdo Jurídico, Brasília-DF: 02 abr. 2021. Disponível em: https://conteudojuridico.com.br/ consulta/Artigos/55222/lei-13-827-2019-altera-a-lei-maria-da-penha-para-permitir-ao-delegado-de-polcia-a-concesso-de-medida-protetiva-de-urgncia. Acesso em: 02 abr. 2021.\nBRASIL. Lei n. 9099, de 26 de setembro de 1995. Dispõe sobre os Juizados Especiais Cíveis e Criminais e dá outras providências. Disponível em: http://www.planalto.gov.br/ccivil_03/leis/l9099.htm. Acesso em: 1 abr. 2021.\nBRASIL. Lei n. 11.340, de 7 de agosto de 2006. Lei Maria da Penha. Disponível em: http://www.planalto.gov.br/ccivil_03/_ato2004-2006/2006/lei/l11340.htm. Acesso em: 1 abr. 2021.\nBRASIL. Lei n. 13.827, de 13 de maio de 2019. Altera a Lei n. 11.340, de 7 de agosto de 2006 (Lei Maria da Penha), para autorizar, nas hipóteses que especifica, a aplicação de medida protetiva de urgência, pela autoridade judicial ou policial, à mulher em situação de violência doméstica e familiar, ou a seus dependentes, e para determinar o registro da medida protetiva de urgência em banco de dados mantido pelo Conselho Nacional de Justiça. Disponível em: http://www.planalto.gov.br/ccivil_03/_Ato2019-2022/2019/Lei/L13827.htm. Acesso em: 1 abril 2021.\nBRASIL. Associação Nacional Dos Membros Do Ministério Público. Nota técnica n. 05/2016/CONAMP. 2016. Disponível em: https://www.conamp.org.br/images/ notas-técnicas/NT_05_PLC%2007_16.pdf. Acesso em: 15 abril 2021.\nCARVALHO, Márcia Haydée Porto de; MAIA, Maicy Milhomem Moscoso. Violência doméstica. Curitiba: Juruá Editora, 2020.\nCUNHA, Rogério Sanches. Lei 13.827/19: Altera a Lei Maria da Penha para permitir a concessão de medida protetiva pela autoridade policial. Revista Meu Site Jurídico, 2019. Disponível em: https://meusitejuridico.editorajuspodivm.com.br/ 2019/05/14/lei-13-82719-altera-lei-maria-da-penha-para-permitir-concessao-de-medida-protetiva-pela-autoridade-policial/. Acesso em: 1 abril 2021.\n______. PINTO, Ronaldo Batista. Violência doméstica: Lei Maria da Penha comentada artigo por artigo. São Paulo: RT, 2007.\nDIAS, Maria Berenice. Medidas protetivas mais protetoras. Disponível em: http://www.mariaberenice.com.br/manager/arq/(cod2_13014)Medidas_protetivas_mais_protetoras.pdf. Acesso em: 16 abr. 2021.\nGARCIA, Thiago. Tudo que você precisa saber sobre: Delegado de Polícia, Lei Maria da Penha e Princípio da Insignificância. São Paulo: Rideel, 2019.\nLEITÃO, Joaquim Jr.; OLIVEIRA, Marcel Gomes de. As implicações da nova Lei n. 13.827/2019: aplicação das medidas protetivas no âmbito da Lei Maria da Penha por delegado de polícia ou de policiais. Revista Jus Navegandi, Teresina, ano 24, n. 5798, mai., 2019. Disponível em: https://jus.com.br/artigos/74012/as-implicacoes-da-nova-lei-n-13-827-2019. Acesso em: 1 abril 2021.\nMONTENEGRO, Marília. Lei Maria da Penha: uma análise criminológica-crítica. Rio de Janeiro: Revan, 2015.\nMÜLLER, Helena Veiga. A (in)constitucionalidade da concessão de medidas protetivas pela autoridade policial: análise no âmbito da violência doméstica. Revista Âmbito Jurídico, 12 set. 2019. Disponível em: https://ambitojuridico.com.br/cadernos/ direito-penal/a-inconstitucionalidade-da-concessao-de-medidas-protetivas-pela-autoridade-policial-analise-no-âmbito-da-violência-doméstica/. Acesso em: 16 abr. 2021.\nSANTOS, Keila Freitas dos. As medidas protetivas da Lei Maria da Penha e sua efetiva aplicabilidade pelo poder público, por ocasião de violência doméstica e familiar contra a mulher. Monografias Brasil Escola. Disponível em: https://monografias.brasilescola.uol.com.br/direito/as-medidas-protetivas-da-lei-maria-da-penha-e-sua-efetiva-aplicabilidade-pelo-poder-publico-por-ocasiao-de-violencia-doméstica-e-familiar-contra-a-mulher.htm. Acesso em: 16 abr. 2021.\nSOUZA, Sérgio Ricardo. Lei Maria da Penha comentada: sob a nova perspectiva dos direitos humanos. 6. ed. Curitiba: Juruá, 2019.\nYOUNG, Pedro. A possibilidade de o delegado conceder medidas protetivas e o direito de defesa. JusBrasil, jun. 2016. Disponível em: https://ppyoung.jus brasil.com.br/artigos/358114778/a-possibilidade-do-delegado-conceder-medidas-protetivas-e-o-direito-de-defesa. Acesso em: 15 abr. 2021.\nIGOR DE ANDRADE BARBOSA: Medida protetiva de urgência/doutrina, 1\nMedida protetiva de urgência/doutrina, 1\nFICHA TÉCNICA // Revista Bonijuris Título original: A Lei 13.827/19 e a possibilidade de aplicação de medidas protetivas de urgência pela autoridade policial: argumentos favoráveis e contrários Title: Law 13.827/19 and the possibility of applying emergency protective measures by the police authority: pros and cons. Autores: Renata Costa do Egyto. Bacharelanda em Direito pelo Centro Universitário Católica do Tocantins (UNICÁTOLICA). E-mail: <EMAIL>. Igor de Andrade Barbosa. Defensor público federal de primeira categoria na Defensoria Pública da União do Tocantins. Professor de Graduação e Pós-Graduação do curso de Direito do Centro Universitário Católica do Tocantins (UNICATÓLICA). E-mail: <EMAIL>. Resumo: Nos 15 anos da vigência da lei Maria da Penha houve significativa elevação de violência doméstica no país. A intervenção do Estado na esfera doméstica pode ser primária ou pós-conflitual. Um dos mecanismos é a medida protetiva de urgência (MPU), voltada ao combate preventivo e repressivo de crimes ultrajantes. Nesse contexto, foi editada a Lei 13.827/19, que trouxe a possibilidade de a autoridade policial conceder MPU, determinando o afastamento do agressor do âmbito doméstico, o que viola o princípio da reserva da jurisdição, pois transfere para a esfera policial atribuições que são do Judiciário, revelando, adicionalmente, o não atendimento a demandas específicas das mulheres vitimadas e ingerência nos direitos fundamentais do acusado. Palavras-chave: AUTORIDADE POLICIAL; MEDIDAS PROTETIVAS DE URGÊNCIA; MULHER; VIOLÊNCIA DOMÉSTICA. Abstract: In the 15 years of the Maria da Penha law, there has been a significant increase in domestic violence in the country. State intervention in the domestic sphere can be primary or post-conflict. One of the mechanisms is the urgent protective measure (MPU), aimed at preventively and repressively combating outrageous crimes. In this context, Law 13.827/19 was enacted, which made it possible for the police authority to grant MPU, determining the removal of the aggressor from the domestic sphere, which violates the principle of reserve of jurisdiction, as it transfers to the police sphere attributions that are of the Judiciary, revealing, additionally, the non-compliance with the specific demands of the victimized women and interference in the fundamental rig\nAUTHORITY; PROTECTIVE URGENT MEASURES; WOMAN; DOMESTIC VIOLENCE. Data de recebimento: 03.05.2021. Data de aprovação: 29.11.2021 Fonte: Revista Bonijuris, vol. 34, n. 1 – #674 – Fev/Mar 2022. Editor: Luiz Fernando de Queiroz, Ed. Bonijuris, Curitiba, PR, Brasil, ISSN 1809-3256 (<EMAIL>).\nNOTAS 1 Art. 2o A assistência social tem por objetivos: I − a proteção social, que visa à garantia da vida, à redução de danos e à prevenção da incidência de riscos, especialmente: a) a proteção à família, à maternidade, à infância, à adolescência e à velhice; b) o amparo às crianças e aos adolescentes carentes; c) a promoção da integração ao mercado de trabalho; d) a habilitação e reabilitação das pessoas com deficiência e a promoção de sua integração à vida comunitária; e e) a garantia de 1 (um) salário-mínimo de benefício mensal à pessoa com deficiência e ao idoso que comprovem não possuir meios de prover a própria manutenção ou de tê-la provida por sua família. II − a vigilância socioassistencial, que visa a analisar territorialmente a capacidade protetiva das famílias e nela a ocorrência de vulnerabilidades, de ameaças, de vitimizações e danos; III – a defesa de direitos, que visa a garantir o pleno acesso aos direitos no conjunto das provisões socioassistenciais. 2 Art. 23. Poderá o juiz, quando necessário, sem prejuízo de outras medidas: I – encaminhar a ofendida e seus dependentes a programa oficial ou comunitário de proteção ou de atendimento; II – determinar a recondução da ofendida e a de seus dependentes ao respectivo domicílio, após afastamento do agressor; III – determinar o afastamento da ofendida do lar, sem prejuízo dos direitos relativos a bens, guarda dos filhos e alimentos; IV – determinar a separação de corpos. V − determinar a matrícula dos dependentes da ofendida em instituição de educação básica mais próxima do seu domicílio, ou a transferência deles para essa instituição, independentemente da existência de vaga. (Incluído pela Lei nº 13.882, de 2019) 3 RE 593.727/MG; HC 107644/SP; MS 23,452/RJ.
Contato: email@exemplo.com
"""
try:
    bruto = chat_clean(texto_teste, max_new_tokens=15000)
    #print("== bruto =="); print(bruto[:1000])
    #print("\n== pós-regras ==")
    print(post_rules(bruto))
except Exception as e:
    print("Falha ao chamar o endpoint:", e)

In [5]:
# Processar corpus no vLLM

from pprint import pprint
files = sorted(glob(SRC_GLOB))
print("Arquivos:", files)

all_stats = []

for src in tqdm(files, desc="Progresso Geral (Arquivos)", position=0, leave=True):
    dst = str(Path(src).with_name(Path(src).stem + OUT_SUFFIX + Path(src).suffix))
    stats = clean_file(src, dst, sample=0)  # use sample>0 para testar
    all_stats.append(stats)

print("\nResumo:")
for s in all_stats:
    pprint(s)

print("Processamento concluído!")

Arquivos: ['corpus_0000.jsonl']


Progresso Geral (Arquivos):   0%|          | 0/1 [00:00<?, ?it/s]
Limpando corpus_0000.jsonl: 0it [00:00, ?it/s]
Limpando corpus_0000.jsonl: 1it [06:20, 380.65s/it]
Limpando corpus_0000.jsonl: 2it [06:51, 174.60s/it]
Limpando corpus_0000.jsonl: 3it [07:09, 103.31s/it]
Limpando corpus_0000.jsonl: 4it [07:14, 64.35s/it] 
Limpando corpus_0000.jsonl: 5it [07:31, 47.28s/it]
Limpando corpus_0000.jsonl: 6it [07:46, 36.58s/it]
Limpando corpus_0000.jsonl: 7it [08:26, 37.40s/it]
Limpando corpus_0000.jsonl: 8it [08:46, 32.12s/it]
Limpando corpus_0000.jsonl: 9it [09:18, 32.11s/it]
Limpando corpus_0000.jsonl: 10it [09:34, 27.04s/it]
Limpando corpus_0000.jsonl: 11it [10:22, 33.29s/it]
Limpando corpus_0000.jsonl: 12it [11:10, 37.76s/it]
Limpando corpus_0000.jsonl: 13it [11:17, 28.68s/it]
Limpando corpus_0000.jsonl: 14it [11:33, 24.63s/it]
Limpando corpus_0000.jsonl: 15it [12:10, 28.48s/it]
Limpando corpus_0000.jsonl: 16it [12:18, 22.34s/it]
Limpando corpus_0000.jsonl: 17it [13:07, 30.40s/it]
Limpando

ERRO INESPERADO em corpus_0000.jsonl: 400 Client Error: Bad Request for url: http://localhost:8000/v1/chat/completions



Limpando corpus_0000.jsonl: 35it [40:20, 60.19s/it]
Limpando corpus_0000.jsonl: 36it [41:55, 70.66s/it]
Limpando corpus_0000.jsonl: 37it [49:16, 181.83s/it]
Limpando corpus_0000.jsonl: 38it [51:47, 172.53s/it]
Limpando corpus_0000.jsonl: 39it [53:24, 149.91s/it]
Limpando corpus_0000.jsonl: 40it [54:03, 116.47s/it]
Limpando corpus_0000.jsonl: 41it [58:12, 156.21s/it]
Limpando corpus_0000.jsonl: 42it [1:00:03, 142.77s/it]
Limpando corpus_0000.jsonl: 43it [1:00:19, 104.59s/it]
Limpando corpus_0000.jsonl: 44it [1:04:08, 142.00s/it]
Limpando corpus_0000.jsonl: 45it [1:04:52, 112.75s/it]
Limpando corpus_0000.jsonl: 46it [1:06:03, 99.97s/it] 
Limpando corpus_0000.jsonl: 47it [1:06:33, 79.15s/it]
Limpando corpus_0000.jsonl: 48it [1:06:58, 62.85s/it]
Limpando corpus_0000.jsonl: 49it [1:08:03, 63.39s/it]
                                                                 
Progresso Geral (Arquivos):   0%|          | 0/1 [1:08:37<?, ?it/s]
Limpando corpus_0000.jsonl: 50it [1:08:37, 54.83s/it]

ERRO INESPERADO em corpus_0000.jsonl: 400 Client Error: Bad Request for url: http://localhost:8000/v1/chat/completions



Limpando corpus_0000.jsonl: 51it [1:08:38, 38.47s/it]
Progresso Geral (Arquivos):   0%|          | 0/1 [1:08:56<?, ?it/s]


KeyboardInterrupt: 